In [1]:
from helper import *

In [2]:
categories = {'pets' : 'familie/haustiere', 'politics' : 'politik',
              'economy' : 'wirtschaft', 'travel' : 'konsum/reisen',
              'health' : 'gesundheit/medizin-krankheit', 'food' : 'ernahrung/lebensmittel',
              'parenting' : 'familie/erziehung'}

In [3]:
for category, path in categories.items():
    print(f'Working with {category} category')

    articles = []
    page_num = 1

    while True:
        url = f'https://www.beobachter.ch/{path}?page={page_num}#page'
        page = requests.get(url)    

        soup = BeautifulSoup(page.content, 'html.parser')
        
        if page_num == 1:
            try:
                limit = int(soup.find('div', class_='_2fQ8qDhH _1deHe_oF').
                            find_all('a', class_='page-loader-btn ShimA1UP')[-1].text)
                print(f'Number of pages = {limit}')
            except AttributeError:
                limit = 1
                
        
        divs = soup.find_all('div', class_='_23Qt4XY-')

        if divs == []:
            break

        for div in divs[1:]:
            sub_divs = div.find_all('a', class_='_2p02b0mw teaser-m-default')

            for sub_div in sub_divs:
                teaser_headline = sub_div.find('div', class_='_2QZpcT0M _3du5gVzi _3iAkatGy')
                teaser = sub_div.find('div', class_='_2QcjsKyb M6MeFidn')

                if None in (teaser_headline, teaser):
                    continue

                teaser_headline = teaser_headline.text.strip()
                teaser = teaser.text.strip()[:-5] #[:-5] to throw away the word "Mehr"
                
                articles.append(teaser_headline)
                articles.append(teaser)

                if len(articles) % 50 == 0:
                    print(f'Scraped {len(articles)} articles')

        page_num += 1
        if page_num > limit:
            break
            
    print(f'Finished working with {category} category. Scraped {len(articles)} articles\n')
    write_to_file(articles, category, 'beobachter_ch')

Working with pets category
Number of pages = 10
Scraped 50 articles
Scraped 100 articles
Scraped 150 articles
Scraped 200 articles
Finished working with pets category. Scraped 214 articles

214 articles of pets category have been scraped from beobachter_ch

______________________________________________________________________________
Working with politics category
Number of pages = 16
Scraped 50 articles
Scraped 100 articles
Scraped 150 articles
Scraped 200 articles
Scraped 250 articles
Scraped 300 articles
Scraped 350 articles
Finished working with politics category. Scraped 362 articles

362 articles of politics category have been scraped from beobachter_ch

______________________________________________________________________________
Working with economy category
Number of pages = 8
Scraped 50 articles
Scraped 100 articles
Scraped 150 articles
Finished working with economy category. Scraped 178 articles

178 articles of economy category have been scraped from beobachter_ch

______